### Extract Text

In [1]:
import fitz  # PyMuPDF

def extract_text_from_pdf(file_path):
    doc = fitz.open(file_path)
    pages = []
    for page_number in range(len(doc)):
        page = doc.load_page(page_number)
        text = page.get_text().strip()
        pages.append({
            "page_number": page_number + 1,
            "text": text
        })
    doc.close()
    return pages

# Extract and print text
pdf_path = "./metformin1.pdf"
pages = extract_text_from_pdf(pdf_path)

### Extract Entities

In [2]:
import spacy
nlp = spacy.load("en_core_web_sm")


In [3]:
# Function to extract entities from text
def extract_entities(text):
    doc = nlp(text)
    entities = []
    for ent in doc.ents:
        entities.append({
            "text": ent.text,
            "label": ent.label_
        })
    return entities

In [5]:
all_entities = []

for page in pages:
    entities = extract_entities(page["text"])
    all_entities.append({
        "page_number": page["page_number"],
        "entities": entities
    })

print(all_entities[0])

{'page_number': 1, 'entities': [{'text': '2', 'label': 'CARDINAL'}, {'text': 'Metformin', 'label': 'PERSON'}, {'text': 'C4H11N5 •', 'label': 'ORG'}, {'text': '165.63', 'label': 'CARDINAL'}, {'text': '12.4', 'label': 'CARDINAL'}, {'text': '1%', 'label': 'PERCENT'}, {'text': '6.68', 'label': 'CARDINAL'}, {'text': '500', 'label': 'CARDINAL'}, {'text': '850', 'label': 'CARDINAL'}, {'text': '1000', 'label': 'CARDINAL'}, {'text': '500', 'label': 'CARDINAL'}, {'text': '850', 'label': 'CARDINAL'}, {'text': '1000', 'label': 'CARDINAL'}, {'text': '500', 'label': 'CARDINAL'}, {'text': '750', 'label': 'CARDINAL'}, {'text': '500', 'label': 'CARDINAL'}, {'text': '4079189', 'label': 'CARDINAL'}, {'text': 'FDA', 'label': 'ORG'}, {'text': 'https://www.fda.gov/drugsatfda', 'label': 'ORG'}]}


In [ ]:
import json


with open("entities.json", "w", encoding="utf-8") as f:
    json.dump(all_entities, f, ensure_ascii=False, indent=2)

### Unique labels

In [8]:
import json

# Load the JSON back in
with open("entities.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Collect all unique labels
unique_labels = {entity["label"] for page in data for entity in page["entities"]}

# Turn into a sorted list if you want them ordered
unique_labels = sorted(unique_labels)

print(unique_labels)

['CARDINAL', 'DATE', 'GPE', 'LAW', 'LOC', 'NORP', 'ORDINAL', 'ORG', 'PERCENT', 'PERSON', 'PRODUCT', 'QUANTITY', 'TIME', 'WORK_OF_ART']


### Building and storing your FAISS index

### Ollama Function

In [15]:
import requests

def ask_ollama(prompt):
    url = "http://home-pc.tail4924f5.ts.net:11434/api/generate"
    headers = {
        "Content-Type": "application/json"
    }
    payload = {
        "model": "llama3",
        "prompt": prompt,
        "stream": False
    }

    response = requests.post(url, headers=headers, json=payload)
    response.raise_for_status()
    
    return response.json()["response"]

In [44]:
ask_ollama("Hello who are you")


"Nice to meet you! I'm LLaMA, a large language model trained by a team of researcher at Meta AI. My primary function is to understand and respond to human input in the form of text or voice. I'm a conversational AI designed to simulate natural language understanding and generation.\n\nI can help with a wide range of topics, from general knowledge and history to science, technology, culture, and more. I can also engage in creative activities like storytelling, poetry, and even humor.\n\nWhat would you like to talk about or ask me?"

In [25]:
import json

# Load the uploaded entities.json file
with open("entities.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Create a set of unique (label, text) pairs
unique_entities = {(ent["label"], ent["text"]) for page in data for ent in page["entities"]}

# Count unique entities
unique_count = len(unique_entities)
unique_count

569

### Dumping nodes in Neo4J

In [20]:
# --- 2. Neo4j Aura connection --
from neo4j import GraphDatabase

NEO4J_URI = "neo4j+s://033a4c34.databases.neo4j.io"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "yQ02-iiANgP8fMDa_Ndj6mq4DioXPbbqGdP2_JBqrLg"
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))


In [21]:
def test_connection():
    try:
        with driver.session() as session:
            result = session.run("RETURN 'Neo4j connection successful!' AS message")
            print(result.single()["message"])
    except Exception as e:
        print("❌ Failed to connect to Neo4j:", e)

test_connection()

Neo4j connection successful!


In [22]:
import json

# --- 1. Load entities.json ---
with open("entities.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# --- 3. Collect unique (label, text) pairs ---
unique_entities = set()
for page in data:
    for ent in page["entities"]:
        label = ent["label"].strip()
        text = ent["text"].strip()
        unique_entities.add((label, text))

# --- 4. Create nodes in Neo4j ---
def create_nodes(tx, entities):
    for label, text in entities:
        # MERGE avoids duplicates if you re-run the script
        query = f"""
        MERGE (n:`{label}` {{text: $text}})
        """
        tx.run(query, text=text)

with driver.session() as session:
    session.write_transaction(create_nodes, unique_entities)

print(f"Inserted {len(unique_entities)} unique nodes.")

driver.close()


/var/folders/35/143126m50gv7zq_ldk50bbhr0000gn/T/ipykernel_36551/2032268651.py:25: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_nodes, unique_entities)


Inserted 568 unique nodes.


## Semantic Chunking

In [41]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def chunk_pages(pages, chunk_size=1000, chunk_overlap=200):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    all_chunks = []
    for page in pages:
        chunks = splitter.split_text(page["text"])
        all_chunks.extend(chunks)
    return all_chunks

In [42]:
chunks = chunk_pages(pages)
for i, chunk in enumerate(chunks, 1):
    print(f"--- Chunk {i} ---")
    print(len(chunk),chunk)
    print()

--- Chunk 1 ---
926 GLUCOPHAGE®  
(metformin hydrochloride) Tablets  
GLUCOPHAGE® XR  
(metformin hydrochloride) Extended-Release Tablets  
DESCRIPTION 
GLUCOPHAGE® (metformin hydrochloride) Tablets and GLUCOPHAGE® XR (metformin 
hydrochloride) Extended-Release Tablets are oral antihyperglycemic drugs used in the 
management of type 2 diabetes. Metformin hydrochloride (N,N-dimethylimidodicarbonimidic 
diamide hydrochloride) is not chemically or pharmacologically related to any other classes of 
oral antihyperglycemic agents. The structural formula is as shown: structural formula
Metformin hydrochloride is a white to off-white crystalline compound with a molecular formula 
of C4H11N5 • HCl and a molecular weight of 165.63. Metformin hydrochloride is freely soluble 
in water and is practically insoluble in acetone, ether, and chloroform. The pKa of metformin is 
12.4. The pH of a 1% aqueous solution of metformin hydrochloride is 6.68.

--- Chunk 2 ---
916 in water and is practically inso

### Making the embeddings of the the labels

In [26]:
import json
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
import pickle

# --- 1. Load entities.json ---
with open("entities.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# --- 2. Extract unique labels ---
labels = sorted({ent["label"] for page in data for ent in page["entities"]})
print("Unique labels:", labels)

# --- 3. Load embedding model ---
model = SentenceTransformer("all-MiniLM-L6-v2")  # Small, fast model

# --- 4. Create embeddings ---
embeddings = model.encode(labels, convert_to_numpy=True, normalize_embeddings=True)

# --- 5. Create FAISS index ---
dim = embeddings.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(embeddings)

# --- 6. Save FAISS index ---
faiss.write_index(index, "labels.index")

# --- 7. Also save mapping from index to labels ---
with open("labels_mapping.pkl", "wb") as f:
    pickle.dump(labels, f)

print("FAISS index and label mapping saved.")


/Users/aaron/Desktop/FragRag/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Unique labels: ['CARDINAL', 'DATE', 'GPE', 'LAW', 'LOC', 'NORP', 'ORDINAL', 'ORG', 'PERCENT', 'PERSON', 'PRODUCT', 'QUANTITY', 'TIME', 'WORK_OF_ART']
FAISS index and label mapping saved.


In [32]:
import json
import faiss
import numpy as np
import pickle
from sentence_transformers import SentenceTransformer

# --- Load entities.json ---
with open("entities.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# --- Extract unique text values ---
unique_texts = sorted({ent["text"].strip() for page in data for ent in page["entities"]})

print(f"Total unique entity texts: {len(unique_texts)}")

# --- Load embedding model ---
model = SentenceTransformer("all-MiniLM-L6-v2")

# --- Create normalized embeddings ---
embeddings = model.encode(unique_texts, convert_to_numpy=True, normalize_embeddings=True)

# --- Create FAISS cosine similarity index ---
dim = embeddings.shape[1]
index = faiss.IndexFlatIP(dim)  # Cosine similarity with normalized vectors
index.add(embeddings)

# --- Save index and mapping ---
faiss.write_index(index, "entity_texts_cosine.index")
with open("entity_texts_mapping.pkl", "wb") as f:
    pickle.dump(unique_texts, f)

print("FAISS index for entity texts saved.")


Total unique entity texts: 563
FAISS index for entity texts saved.


### Searching the Faiss

In [ ]:
# --- Load index and mapping ---
index = faiss.read_index("entity_texts_cosine.index")
with open("entity_texts_mapping.pkl", "rb") as f:
    unique_texts = pickle.load(f)

# --- Load embedding model ---
model = SentenceTransformer("all-MiniLM-L6-v2")

# --- Query ---
query = """916 in water and is practically insoluble in acetone, ether, and chloroform. The pKa of metformin is 
12.4. The pH of a 1% aqueous solution of metformin hydrochloride is 6.68. 
GLUCOPHAGE tablets contain 500 mg, 850 mg, or 1000 mg of metformin hydrochloride. Each 
tablet contains the inactive ingredients povidone and magnesium stearate. In addition, the coating 
for the 500 mg and 850 mg tablets contains hypromellose and the coating for the 1000 mg tablet 
contains hypromellose and polyethylene glycol. 
GLUCOPHAGE XR contains 500 mg or 750 mg of metformin hydrochloride as the active 
ingredient."""
embedding = model.encode([query], convert_to_numpy=True, normalize_embeddings=True)

# --- Search ---
similarities, indices = index.search(embedding, k=30)
results = [(unique_texts[idx], float(sim)) for idx, sim in zip(indices[0], similarities[0])]

# --- Sort descending (already sorted by FAISS) ---
for text, sim in results:
    print(f"{text} -> {sim:.4f}")


Twice Daily 
GLUCOPHAGE XR 
1000 -> 0.5015
358 L. Metformin -> 0.4670
600 mg/kg -> 0.4543
45 mL/min/1.73 m2 -> 0.4325
Metformin -> 0.4306
mL. Metformin -> 0.4291
GLUCOPHAGE -> 0.4197
60 mL -> 0.3668
Gender 
Metformin -> 0.3663
Total Triglycerides -> 0.3614
Dosage -> 0.3463
DOSAGE -> 0.3463
30 mL -> 0.3352
50 grams -> 0.3273
mcg/mL -> 0.3174
Plasma Glucose -> 0.3143
Serum Lipid 
Variables -> 0.3094
https://www.fda.gov/drugsatfda -> 0.2962
HDL-Cholesterol -> 0.2759
Insulin Dose -> 0.2659
GLU -> 0.2565
Hemoglobin A1c -> 0.2547
LDL-Cholesterol -> 0.2527
Drug Interactions -> 0.2470
serum creatinine -> 0.2396
CLINICAL PHARMACOLOGY -> 0.2247
9.6 
2.6 
Nausea/Vomiting 
6.5 
1.5 -> 0.2213
Once Daily 
Hemoglobin A1c -> 0.2159
Placebo/Insulin Summary -> 0.1946
Bristol-Myers Squibb Company -> 0.1914


### Building Relationships

In [49]:
import faiss
import pickle
import json
from sentence_transformers import SentenceTransformer

def build_relationships_for_chunks(chunks, k=30, save_path="relationships.json"):
    # --- Load FAISS index and mapping ---
    index = faiss.read_index("entity_texts_cosine.index")
    with open("entity_texts_mapping.pkl", "rb") as f:
        unique_texts = pickle.load(f)

    # --- Load embedding model ---
    model = SentenceTransformer("all-MiniLM-L6-v2")

    all_relationships = []

    for chunk in chunks:
        # Get relevant entities from FAISS
        embedding = model.encode([chunk], convert_to_numpy=True, normalize_embeddings=True)
        similarities, indices = index.search(embedding, k=k)
        results = [unique_texts[idx] for idx in indices[0]]

        # Prepare the prompt
        prompt = f"""
You are an expert in knowledge graph extraction.
Given the following text chunk and list of relevant entities, 
return ONLY a valid JSON array describing entity relationships found in the chunk.
No explanations, no commentary — ONLY the JSON array.
Format:
[
  {{"source": "<entity1>", "relationship": "<relation>", "target": "<entity2>"}}
]

Text chunk:
\"\"\"{chunk}\"\"\"

Relevant entities:
{results}

Respond with the JSON array only.
"""

        llm_output = ask_ollama(prompt).strip()  # Uses your already defined ask_ollama()

        try:
            relationships = json.loads(llm_output)
            if isinstance(relationships, list):
                all_relationships.extend(relationships)
            else:
                print("⚠️ LLM returned JSON but not a list:", llm_output)
        except json.JSONDecodeError:
            print("❌ JSON parse error. LLM output was:")
            print(llm_output)

        # Save progress after each chunk
        with open(save_path, "w", encoding="utf-8") as f:
            json.dump(all_relationships, f, ensure_ascii=False, indent=2)

    return all_relationships


In [50]:
build_relationships_for_chunks(chunks)

❌ JSON parse error. LLM output was:
[
  {"source": "Metformin", "relationship": "associated", "target": "lactic acidosis"},
  {"source": "Renal Impairment", "relationship": "risk factor", "target": "metformin-associated lactic acidosis"},
  {"source": "topiramate", "relationship": "concomitant use", "target": "metformin"},
  {"source": "age 65 years old or greater", "relationship": "risk factor", "target": "metformin-associated lactic acidosis"},
  {"source": "surgery and other procedures", "relationship": "risk factor", "target": "metformin-associated lactic acidosis"},
  {"source": "hypoxic states (e.g., acute congestive heart failure)", "relationship": "risk factor", "target": "metformin-associated lactic acidosis"},
  {"source": "excessive alcohol intake", "relationship": "risk factor", "target": "metformin-associated lactic acidosis"},
  {"source": "hepatic impairment", "relationship": "risk factor", "target": "metformin-associated lactic acidosis"}
❌ JSON parse error. LLM output 

[{'source': 'GLUCOPHAGE', 'relationship': 'relatedTo', 'target': 'Metformin'},
 {'source': 'Metformin',
  'relationship': 'hasFormula',
  'target': 'C4H11N5 • HCl'},
 {'source': 'metformin', 'relationship': 'contains', 'target': 'povidone'},
 {'source': 'metformin',
  'relationship': 'contains',
  'target': 'magnesium stearate'},
 {'source': 'GLUCOPHAGE tablets',
  'relationship': 'contain',
  'target': 'metformin hydrochloride'},
 {'source': 'GLUCOPHAGE XR',
  'relationship': 'contain',
  'target': 'metformin hydrochloride'},
 {'source': 'sodium carboxymethyl cellulose',
  'relationship': 'ingredient',
  'target': 'GLUCOPHAGE XR'},
 {'source': 'hypromellose',
  'relationship': 'ingredient',
  'target': 'GLUCOPHAGE XR'},
 {'source': 'magnesium stearate',
  'relationship': 'ingredient',
  'target': 'GLUCOPHAGE XR'},
 {'source': 'metformin hydrochloride',
  'relationship': 'component',
  'target': 'GLUCOPHAGE XR'},
 {'source': 'polymer matrix system',
  'relationship': 'component',
  'ta

In [57]:
import re
import json
from neo4j import GraphDatabase

NEO4J_URI = "neo4j+s://033a4c34.databases.neo4j.io"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "yQ02-iiANgP8fMDa_Ndj6mq4DioXPbbqGdP2_JBqrLg"
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

def sanitize_relationship(rel):
    # Uppercase and replace all non-alphanumeric characters with underscores
    clean = re.sub(r'[^A-Z0-9_]', '_', str(rel).upper())
    return clean if clean else "RELATED_TO"  # default if empty

def create_relationships_from_json(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        relationships = json.load(f)

    with driver.session() as session:
        for rel in relationships:
            source = rel.get("source")
            relationship = sanitize_relationship(rel.get("relationship", "RELATED_TO"))
            target = rel.get("target")

            if not source or not target:
                print(f"⚠️ Skipping invalid relationship: {rel}")
                continue

            query = f"""
            MATCH (a {{text: $source}}), (b {{text: $target}})
            MERGE (a)-[r:{relationship}]->(b)
            """
            session.run(query, source=source, target=target)

    print(f"✅ Created {len(relationships)} relationships from {file_path}")


In [58]:
create_relationships_from_json("relationships.json")

⚠️ Skipping invalid relationship: {'source': 'Metformin', 'relationship': '', 'target': ''}
⚠️ Skipping invalid relationship: {'source': 'FDA', 'relationship': '', 'target': ''}
⚠️ Skipping invalid relationship: {'source': 'GLUCOPHAGE', 'relationship': '', 'target': ''}
⚠️ Skipping invalid relationship: {'source': 'Twice Daily \nGLUCOPHAGE XR \n1000', 'relationship': '', 'target': ''}
⚠️ Skipping invalid relationship: {'source': 'Plasma Glucose', 'relationship': '', 'target': ''}
⚠️ Skipping invalid relationship: {'source': 'GLUCOPHAGE XR', 'relationship': '', 'target': ''}
⚠️ Skipping invalid relationship: {'source': 'HDL-Cholesterol', 'relationship': '', 'target': ''}
⚠️ Skipping invalid relationship: {'source': 'GLUCOPHAGE XR', 'relationship': '', 'target': ''}
⚠️ Skipping invalid relationship: {'source': 'GLUCOPHAGE', 'relationship': '', 'target': ''}
⚠️ Skipping invalid relationship: {'source': 'GLUCOPHAGE XR', 'relationship': 'antidiabetic drug', 'target': None}
⚠️ Skipping inval

### Querying the KG

qs: ingestion of what medicine causes diarrhea

In [83]:
def query_ent_ext(query):
    embedding = model.encode([query], convert_to_numpy=True, normalize_embeddings=True)

    retrieved_ent=[]
    # --- Search ---
    similarities, indices = index.search(embedding, k=5)
    results = [(unique_texts[idx], float(sim)) for idx, sim in zip(indices[0], similarities[0])]

    # --- Sort descending (already sorted by FAISS) ---
    for text, sim in results:
        retrieved_ent.append(text)
    return retrieved_ent

answer = query_ent_ext(query = "Ingestion of what medicine cause diarrhea")
answer


['Diarrhea',
 'Drug Interactions',
 'CLINICAL PHARMACOLOGY',
 'Nifedipine',
 'https://www.fda.gov/drugsatfda']

In [91]:
def get_entities_with_relationships(driver, entity_texts):
    query = """
    WITH $entityList AS entityList
    MATCH (n)
    WHERE n.text IN entityList
    OPTIONAL MATCH (n)-[r]-(m)
    RETURN {
        node: {
            label: labels(n),
            text: n.text
        },
        relationship: CASE WHEN r IS NOT NULL THEN {
            type: type(r),
            properties: properties(r)
        } ELSE null END,
        connectedNode: CASE WHEN m IS NOT NULL THEN {
            label: labels(m),
            text: m.text
        } ELSE null END
    } AS result
    """

    with driver.session() as session:
        results = session.run(query, entityList=entity_texts)
        structured_output = [record["result"] for record in results]
    
    return structured_output

answer = get_entities_with_relationships(driver,query_ent_ext("is glucophage related to body weight"))

print(answer)

[{'relationship': {'properties': {}, 'type': 'TREATS'}, 'node': {'text': 'GLUCOPHAGE', 'label': ['ORG']}, 'connectedNode': {'text': 'Hemoglobin A1c', 'label': ['PERSON']}}, {'relationship': {'properties': {}, 'type': 'RELATED_TO'}, 'node': {'text': 'GLUCOPHAGE', 'label': ['ORG']}, 'connectedNode': {'text': 'Hemoglobin A1c', 'label': ['PERSON']}}, {'relationship': {'properties': {}, 'type': 'RELATED_TO'}, 'node': {'text': 'GLUCOPHAGE', 'label': ['ORG']}, 'connectedNode': {'text': 'Body Weight', 'label': ['PERSON']}}, {'relationship': {'properties': {}, 'type': 'RELATED_TO'}, 'node': {'text': 'GLUCOPHAGE', 'label': ['ORG']}, 'connectedNode': {'text': 'Plasma Glucose', 'label': ['PERSON']}}, {'relationship': {'properties': {}, 'type': 'RELATED_TO'}, 'node': {'text': 'GLUCOPHAGE', 'label': ['ORG']}, 'connectedNode': {'text': 'GLU', 'label': ['ORG']}}, {'relationship': {'properties': {}, 'type': 'RELATED_TO'}, 'node': {'text': 'GLUCOPHAGE', 'label': ['ORG']}, 'connectedNode': {'text': 'Diar

In [94]:
import json

def answer_question_with_graph(driver, question):
    # Step 1: Fetch graph data
    entity_list=query_ent_ext(question)
    graph_data = get_entities_with_relationships(driver, entity_list)

    # Step 2: Build a prompt for the LLM
    prompt = f"""
You are a reasoning assistant working with a knowledge graph.

The graph is given below as JSON. 
It contains:
- "node" = the starting node requested
- "relationship" = the exact relationship (type + properties) between the node and another node
- "connectedNode" = the node at the other end of the relationship

STRICT INSTRUCTIONS:
1. Only use relationships that are explicitly shown in the JSON. 
2. Do NOT make any guesses, assumptions, or use outside knowledge.
3. If the answer cannot be found in the relationships, say: "No direct answer can be found from the provided graph."
4. Always show the exact chain of relationships from the graph that led to your answer.
5. If multiple paths exist, show each path separately.
6. If no relationship connects the queried entities, say so clearly.

Graph Data:
{json.dumps(graph_data, indent=2)}

Question: {question}

Your response should be in the following format:

Answer: <Direct answer based only on relationships in the graph>
Relationships used:
1. <Node A> -[RELATIONSHIP_TYPE]-> <Node B>
2. ...

In the end gather whatever you founf out and give a paragraph answer.
"""

    print(graph_data)

    # Step 3: Send to Ollama
    return ask_ollama(prompt)


In [95]:
# Example usage
# question = "ingestion of what medicine causes diarrhea"
question = "What all is mentioned about Metformin?"
response = answer_question_with_graph(driver, question)

print(response)

[{'relationship': None, 'node': {'text': '358 L. Metformin', 'label': ['PERSON']}, 'connectedNode': None}, {'relationship': None, 'node': {'text': 'CLINICAL PHARMACOLOGY', 'label': ['PERSON']}, 'connectedNode': None}, {'relationship': None, 'node': {'text': 'mL. Metformin', 'label': ['PERSON']}, 'connectedNode': None}, {'relationship': None, 'node': {'text': 'Gender \nMetformin', 'label': ['ORG']}, 'connectedNode': None}, {'relationship': {'properties': {}, 'type': 'RELATED_TO'}, 'node': {'text': 'Metformin', 'label': ['PERSON']}, 'connectedNode': {'text': 'Renal Impairment', 'label': ['PERSON']}}, {'relationship': {'properties': {}, 'type': 'RELATEDTO'}, 'node': {'text': 'Metformin', 'label': ['PERSON']}, 'connectedNode': {'text': 'GLUCOPHAGE', 'label': ['ORG']}}, {'relationship': {'properties': {}, 'type': 'IS_INCREASED'}, 'node': {'text': 'Metformin', 'label': ['PERSON']}, 'connectedNode': {'text': 'Cmax', 'label': ['PERSON']}}, {'relationship': {'properties': {}, 'type': 'ASSOCIATE